# <center>Miniproyecto 7</center>
## <center>Modelación y Simulación</center>
<center>Javier Anleu Alegría - 17149</center>

<center>Andrea Argüello - 17801</center>

## Librerías

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import display, Latex

## Función inversa de distribución exponencial, E[X] y Var(X)

In [2]:
exponentialInv = lambda x,lam: -np.log(1-x)/lam # donde x es U(0,1)
mu_exponencial = lambda lam: 1/lam
sigma_cuadrado_exponencial = lambda lam: 1/(lam**2)

### Ejercicio 1: *Variables antitéticas*, $\lambda = 1$ 

Calcular el valor esperado de una variable aleatoria exponencial con $\lambda =1$, utilizando el método de variables anititéticas.

**Parámetro de la distribución exponencial:**

In [3]:
lambda_1 = 1

**Función para simular con $n$ repeticiones la distribución exponencial**

In [4]:
def expAntithetic(repeticiones):
    X1 = []
    X2 = []
    Y = []
    for i in range(repeticiones):
        x = 1
        while x == 1:
            x = random.uniform(0,1)
        X1.append(exponentialInv(x,lambda_1))
        X2.append(exponentialInv(1-x,lambda_1))
        Y.append((X1[-1]+X2[-1])/2)
    return X1, X2, Y

**Corroboración de los supuestos de la simulación por variables antitéticas**

**Funciones para determinar las varianzas de las variables**

In [5]:
cov_xy = lambda X, Y: np.mean([x * y for x, y in zip(X, Y)])-np.mean(X)*np.mean(Y)

In [6]:
var = lambda X: np.mean([x**2 for x in X])-(np.mean(X))**2

In [7]:
var_mean_xy = lambda X, Y: (var(X) + var(Y) + 2*cov_xy(X,Y))/4

**Simulación de la distribución exponencial con $n = 10,000$**

Fijación del *seed*

In [8]:
random.seed(500)

In [9]:
X1, X2, Y = expAntithetic(10000)

In [10]:
display(Latex(f'$Cov(X_1,X_2)$ = %f' % (cov_xy(X1,X2))))

<IPython.core.display.Latex object>

Queda demostrado que la covarianza entre $X_1$ y $X_2$ es negativa. Como ambas provienen de la distribución exponencial, entonces $X_1$ y $X_2$ son antitéticas.

In [11]:
display(Latex(f'$E[Y]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(Y), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [12]:
display(Latex(f'$Var(Y)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(Y), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

### Ejercicio 2: *Variables de control*, $\lambda = 1$ 

Calcular el valor esperado de una variable aleatoria exponencial con $\lambda =1$, utilizando el método de variables de control.

**Funciones para simular con $n$ repeticiones la distribución exponencial y determinar el estimado de $c^*$**

In [13]:
def ej2(repeticiones):
    X = []
    Y = []
    for i in range(repeticiones):
        x = 1
        y = 1
        while x == 1:
            x = random.uniform(0,1)
        while y == 1:
            y = random.uniform(0,1) # sabemos que mu_y = 0.5 y var_y = 1/12
        X.append(exponentialInv(x,lambda_1))
        Y.append(y)
    return X, Y

def c_asterisco(lista1,lista2): #Basado en el ejemplo visto en clase
    n=0
    d=0
    promedio_x=np.mean(lista1)
    promedio_y=np.mean(lista2)
    for i in range(len(lista1)):
        n=n+(lista1[i]-promedio_x)*(lista2[i]-promedio_y)
        d=d+pow(lista2[i]-promedio_y,2)
    return -1*n/d

**Simulación de la distribución exponencial con $n=10,000$**

Fijación del *seed*

In [14]:
random.seed(487)

In [15]:
# A esta, le pasamos el valor teórico de la media de la distribución conocida Y, mu_y
control = lambda X, Y, mu_y, c: [x + c*(y - mu_y) for x, y in zip(X,Y)]

X, Y = ej2(10000)

c_min = c_asterisco(X, Y)

In [16]:
display(Latex(f'$E[X]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(control(X, Y, 0.5, c_min)), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [17]:
display(Latex(f'$Var(X)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(control(X, Y, 0.5, c_min)), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

## Ejercicio 3: Método estratificado, $\lambda = 1$ 

Calcular el valor esperado de una variable aleatoria exponencial con $\lambda = 1$, utilizando el método estratificado. Se escogen tres estratos (intervalos): $[0,1)$, $[1,3)$ y $[3,\infty)$.

*El problema fue parcialmente resuelto utilizando el método de estratificación descrito en: https://jjtejada.files.wordpress.com/2014/01/stratified_sampling.pdf*

**Probabilidades de obtener una observación en cada uno de los estratos**

Ya que se utiliza la distribución continua uniforme estándar ($\mathscr{U} (0,1)$) para generar las variables aleatorias de la distribución, se puede determinar la probabilidad $P \Big(X \in S_k\Big) = p_k$ evaluando las probabilidades de $X\in S_k$ en la distribución:

*Para el estrato $S_1$, donde $X\in [0,1)$*
$$p_1 = P \Big(X \in S_1\Big) = \int_0^1 \lambda e^{-\lambda x} dx = 1 - e^{-1} \approx 0.632120$$

*Para el estrato $S_2$, donde $X\in [1,3)$*
$$p_2 = P \Big(X \in S_2\Big) = \int_1^3 \lambda e^{-\lambda x} dx = -e^{-3} + e^{-1} \approx 0.318092$$

*Para el estrato $S_3$, donde $X\in [3,\infty)$*
$$p_3 = P \Big(X \in S_3\Big) = \int_3^\infty \lambda e^{-\lambda x} dx = 0 + e^{-3} \approx 0.049787$$

De esta manera, cuando se generen los números aleatorios por $\mathscr{U} (0,1)$, se obtiene:

$$\text{Si } 0 \leq \mathscr{U} (0,1) < 0.632120 \rightarrow X \in [0,1)$$

$$\text{Si } 0.632120 \leq \mathscr{U} (0,1) < 0.950213 \rightarrow X \in [1,3)$$

$$\text{Si } 0.950213 \leq \mathscr{U} (0,1) \leq 1 \rightarrow X \in [3,\infty)$$

**Determinación del número de observaciones por cada estrato**

Para obtener la reducción de varianza, debe considerarse el número de observaciones por cada uno de los estratos $n_k$, el cual se puede determinar por medio de la siguiente ecuación:

$$n_k = \Big(\frac{p_k \sigma_k}{\sum _{k=1}^K p_k \sigma_k}\Big)n$$

Donde $n$ corresponde al total de observaciones que se utilizarán para la simulación. Ya que se desconoce $\sigma_k$, este se puede estimar por medio de una simulación simple. En este caso, se generarán números aleatorios de $\mathscr{U} (0,1)$ hasta obtener 10000 de cada estrato, y a partir de la varianza de estas muestras, se determinará el número de observaciones que cada estrato deberá tener para la simulación. Para la simulación final, $n = 10,000$.

*Número de observaciones para la simulación final:*

In [18]:
n = 10**4

Fijación del *seed*

In [19]:
random.seed(np.pi)

In [20]:
x_1 = []
x_2 = []
x_3 = []
while (len(x_1)<10000) or (len(x_2)<10000) or (len(x_3)<10000):
    u = random.uniform(0,1)
    if (u < 0.63212) and (len(x_1)<10000):
        x_1.append(exponentialInv(u,1))
    elif (u >= 0.63212 and u < 0.950213) and (len(x_2)<10000):
        x_2.append(exponentialInv(u,1))
    elif (len(x_3)<10000):
        x_3.append(exponentialInv(u,1))
        
sigmas = []
sigmas.append(np.var(x_1, ddof = 1)**0.5)
sigmas.append(np.var(x_2, ddof = 1)**0.5)
sigmas.append(np.var(x_3, ddof = 1)**0.5)
p_k = [0.632120, 0.318092, 0.049787]

def n_kCalc(n,sigmas, p_k):
    total = 0
    for i in range(len(sigmas)):
        total += p_k[i]*sigmas[i]
    n_k = [int(np.round(n*(p_k[i]*sigmas[i]/total))) for i in range(len(sigmas))]
    return n_k

n_k = n_kCalc(n, sigmas, p_k)
for i in range(len(n_k)):
    display(Latex("$n_"+str(i+1)+"$: "+str(n_k[i])))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

**Cálculo de los estimadores**

Una vez se determinaron los números de observaciones de los estratos, se puede proceder a la simulación como tal. En este caso, se obtendrá el estimador del valor esperado por medio de un promedio ponderado de los estratos, tal que:

$$\theta = \sum _{k=1}^{K} p_k \Big[\frac{1}{n_k}\sum_{i=1}^{n_k}f(X_i^k)\Big] = \sum _{k=1}^{K} p_k \overline{f}_{n_k}^k$$

De manera similar, la varianza se calcula:

$$\sigma^2 = \sum _{k=1}^{K} \frac{p_k^2 \sigma^2}{n_k}$$

Sin embargo, ya que se desconoce la varianza poblacional de los estratos, se utiliza $S_{k,n_k}^2$, tal que:
$$S_{k,n_k}^2 = \frac{1}{n_k -1} \sum_{i=1}^{n_k} \Big(f(X_i^k) - \overline{f}_{n_k}^k\Big)^2$$

Por lo tanto:

$$\hat{\sigma}^2 = \sum _{k=1}^{K} \frac{p_k^2 S_{k,n_k}^2}{n_k}$$

**Simulación de la distribución exponencial con $n=10,000$**

*Fijación del seed:*

In [21]:
random.seed(2*np.pi)

In [22]:
s_1 = []
s_2 = []
s_3 = []

while (len(s_1)<n_k[0]):
    s_1.append(exponentialInv(random.uniform(0,p_k[0]), 1))
    
while (len(s_2)<n_k[1]):
    s_2.append(exponentialInv(random.uniform(p_k[0],p_k[0]+p_k[1]), 1))    

while (len(s_3)<n_k[2]):
    s_3.append(exponentialInv(random.uniform(p_k[0]+p_k[1],1), 1))
        
mean_cells = []
mean_cells.append(np.mean(s_1))
mean_cells.append(np.mean(s_2))
mean_cells.append(np.mean(s_3))

sigma_cells = []
sigma_cells.append(np.var(s_1, ddof = 1))
sigma_cells.append(np.var(s_2, ddof = 1))
sigma_cells.append(np.var(s_3, ddof = 1))

theta = 0
for i in range(len(mean_cells)):
    theta += p_k[i]*mean_cells[i]
    
sigma = 0
for i in range(len(sigma_cells)):
    sigma += ((p_k[i]**2)*(sigma_cells[i])**2)/n_k[i]
    
display(Latex("$\epsilon$: "+str(theta) + "; $\epsilon$ esperado: " + str(mu_exponencial(lambda_1))), Latex("$\hat{\sigma}^2$: "+str(sigma) + "; $\sigma^2$ esperado: " + str(sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [27]:
display(Latex(f'$\gamma$'))

<IPython.core.display.Latex object>

### Ejercicio 4

Sea $X$ variable aleatoria exponencial con media 1; y dado $X=x$, $Y$ es una variable aleatoria exponencial con media $x$ ($X$ y $Y$ son variables aleatorias dependientes). Dé una forma eficiente para estimar $P(XY\leq3)$.

Nótese que si $X$ y $Y$ son exponenciales, y $E[X]=1$ y $E[Y|X=x]=x$, como el valor esperado de una variable aleatoria exponencial es $\frac{1}{\lambda}$, $\lambda_X = 1$ y $\lambda_Y = \frac{1}{x}$ para $X=x$.

**Función para simular con $n$ repeticiones la distribución exponencial**

In [23]:
def ej4(repeticiones):
    X=[]
    Y=[]
    for i in range(repeticiones):
        x=1
        y=1
        while x==1:
            x=random.uniform(0,1)
        X.append(exponentialInv(x,lambda_1))
        while y==1:
            y=random.uniform(0,1)
        Y.append(exponentialInv(y,1/X[-1]))
    return X,Y

**Simulación de $X$ y $Y$ con $n = 10,000$**

In [24]:
random.seed(5050)

In [25]:
repeat = 10000
X, Y = ej4(repeat)

**Estimación de $P(XY\leq3)$**

In [26]:
total = [0]*repeat
for x in range(len(X)):
    if X[x]*Y[x] <= 3:
        total[x]=1
display(Latex(f'$P(XY\leq3)$ obtenida: %f' % (np.mean(total))))

<IPython.core.display.Latex object>